In [6]:
import mne
import matplotlib.pyplot as plt
import numpy as np
from mne_features.feature_extraction import extract_features,FeatureExtractor
from mne_features.univariate import compute_pow_freq_bands
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
mne.set_log_level('WARNING')
from tqdm import tqdm
import os
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler

In [2]:
metric="f1"
root_path=r'E:\ZKP\persons comparision\Band Analysis\Epoch\Epoch_Super'
bands=['delta','theta','alpha','beta','gamma',"all"]
drug_path_before=os.path.join(root_path,'drug_before')
drug_path_before_s2=os.path.join(root_path,'drug_before_s2')
health_path=os.path.join(root_path,'health')
health_path_s2=os.path.join(root_path,'health_s2')
drug_path_after=os.path.join(root_path,'drug_after')
drug_path_after_s2=os.path.join(root_path,'drug_after_s2')

# 
# health_after=os.path.join(root_path,'health_after')
# health_compare=os.path.join(root_path,'health_for_compare')

In [3]:
def train_RF(X_,y_,):
    clf_ = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=20))
    accuracy_scores = []
    f1_scores = []
    auc=[]
    result=[]
    for i,(train_index, test_index) in enumerate((StratifiedKFold(n_splits=5, shuffle=True).split(X_, y_))):
        X_train, X_test = X_[train_index], X_[test_index]
        y_train, y_test = y_[train_index], y_[test_index]
        clf_.fit(X_train,y_train)
        y_pred=clf_.predict(X_test)
        accuracy_scores.append(accuracy_score(y_test,y_pred))
        f1_scores.append(f1_score(y_test,y_pred))
        auc.append(roc_auc_score(y_test,y_pred))
    result.append([np.mean(accuracy_scores),np.std(accuracy_scores)])
    result.append([np.mean(f1_scores),np.std(f1_scores)])
    result.append([np.mean(auc),np.std(auc)])
    df=pd.DataFrame(result,columns=['mean','std'],index=['accuracy','f1','roc_auc']).T
    return df,clf_


In [4]:
def run_algo(path_1,path_2):
    all_measure=[]
    Overall_X=[]
    Overall_y=None
    for band in tqdm(bands[:-1]):
        drug_data=pd.read_excel(os.path.join(path_1,band+'.xlsx'))  
        health_data=pd.read_excel(os.path.join(path_2,band+'.xlsx'))
        # drug_data.drop(columns=['ID'],inplace=True)
        # health_data.drop(columns=['ID'],inplace=True)
        X=np.concatenate((drug_data.values,health_data.values),axis=0)
        y=np.concatenate((np.ones(drug_data.shape[0]),np.zeros(health_data.shape[0])),axis=0)
        Overall_X.append(X)
        if Overall_y is None:
            Overall_y=y
        df,_=train_RF(X,y)
        mean_roc=df[metric].to_list()[0]
        std_roc=df[metric].to_list()[1]
        all_measure.append("{:.2f}({:.2f})".format(mean_roc,std_roc))
    Overall_X=np.mean(Overall_X,axis=0)
    df,_=train_RF(Overall_X,Overall_y)
    mean_roc=df[metric].to_list()[0]
    std_roc=df[metric].to_list()[1]
    all_measure.append("{:.2f}({:.2f})".format(mean_roc,std_roc))
    return all_measure
b_h=run_algo(drug_path_before,health_path)
a_h=run_algo(drug_path_after,health_path)
a_b=run_algo(drug_path_after,drug_path_before)
b1_b2=run_algo(drug_path_before_s2,drug_path_before)
a1_a2=run_algo(drug_path_after,drug_path_after_s2)
h1_h2=run_algo(health_path,health_path_s2)
hs2_bs2=run_algo(drug_path_before_s2,health_path_s2)
as2_hs2=run_algo(drug_path_after_s2,health_path_s2)
as2_bs2=run_algo(drug_path_after_s2,drug_path_before_s2)
# 
# hb_ha=run_algo(health_compare,health_after)

100%|██████████| 5/5 [00:13<00:00,  2.80s/it]


In [5]:
pd.DataFrame([b_h,a_h,a_b,b1_b2,a1_a2,h1_h2,hs2_bs2,as2_hs2,as2_bs2],columns=bands[:],index=['Before TMS v.s Health','After TMS v.s Health','After TMS v.s Before TMS','Before TMS(S1) v.s Before TMS(S2)','After TMS(S1) v.s After TMS(S2)','Health(S1) v.s Health(S2)','Drug(S2) v.s Health(S2)','Drug_after(S2) v.s Health(S2)','Drug(S2) v.s Drug-after(S2)'])

,delta,theta,alpha,beta,gamma,all
Before TMS v.s Health,0.82(0.03),0.75(0.05),0.80(0.03),0.88(0.02),0.90(0.02),0.87(0.02)
After TMS v.s Health,0.80(0.02),0.71(0.04),0.78(0.04),0.82(0.06),0.88(0.01),0.83(0.03)
After TMS v.s Before TMS,0.74(0.03),0.62(0.05),0.61(0.03),0.77(0.04),0.84(0.03),0.77(0.02)
Before TMS(S1) v.s Before TMS(S2),0.50(0.03),0.53(0.06),0.49(0.05),0.44(0.05),0.48(0.04),0.51(0.06)
After TMS(S1) v.s After TMS(S2),0.53(0.03),0.51(0.03),0.52(0.04),0.51(0.03),0.49(0.04),0.54(0.04)
Health(S1) v.s Health(S2),0.52(0.05),0.49(0.03),0.51(0.04),0.55(0.07),0.47(0.03),0.50(0.05)
Drug(S2) v.s Health(S2),0.74(0.04),0.75(0.03),0.74(0.03),0.82(0.04),0.88(0.04),0.80(0.06)
Drug_after(S2) v.s Health(S2),0.73(0.03),0.71(0.02),0.77(0.03),0.79(0.04),0.87(0.03),0.81(0.04)
Drug(S2) v.s Drug-after(S2),0.71(0.05),0.65(0.03),0.60(0.03),0.75(0.03),0.80(0.02),0.76(0.03)
